# Pre-processamento dos dados

## Imports e infos do dataset

In [ ]:
import os
import joblib
import pandas as pd
from sklearn.cluster import OPTICS
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
df = pd.read_csv("../data/raw/bq-results-20250630-233421-1751326537000.csv")

# 1. Visão geral
print(df.info())
print("==" * 40)

# 2. Missing values
print("Valores nulos: ", df.isna().sum())
print("==" * 40)
print("Valores nulos p/col: ", df.isna().mean().sort_values(ascending=False))
print("==" * 40)

# 3. Informações gerais
print("Total de linhas:", df.shape[0])
print("Texto escrito no card:", df["card_written_text"].nunique())
print("Texto falado do card:", df["card_spoken_text"].nunique())
print("Localização do click:", df["click_location"].nunique())
print("Quantidade de usuarios:", df["user_uuid"].nunique())


## Limpar colunas principais nulas

As principais colunas para o estudo são:

- user_uuid (Identificador único dos usuários)
- click_location (Localização do click na tela em coordenadas)
- card_written_text (Texto escrito no cartão/pictograma)
- event_timestamp (Registro de data e hora do click)

In [ ]:
# Define a lista de colunas que não podem ter valores nulos
main_columns = ['user_uuid', 'click_location', 'card_written_text', 'event_timestamp']

# Remove qualquer LINHA que tenha valor nulo em QUALQUER uma das colunas da lista acima
df_clean = df.dropna(subset=main_columns).copy()

print("Total de linhas:", df.shape[0])
print("Total de linhas após limpeza:", df_clean.shape[0])

## Criação de colunas com base no horário

As principais colunas para o estudo são:

- week_day (indica o dia da semana (Monday, Tuesday, Wednesday,
Thursday, Friday, Saturday, Sunday))
- hour (Representa as horas do dia (de 0 a 23))
- year_num (Utilizado para a separação dos
dados por número de semana.)
- week_num (Representa o número da semana (1 a 52))
- week_order (Calculada a partir das colunas
week_num e year_num, atribuindo valores em ordem crescente)
- period_day (O período do dia em que aconteceu a interação, sendo
gerada a partir da coluna hour)

### Horários por período:
- midnight: (0-5h)
- dawn: (6-8h)
- morning: (9-11h)
- noon: (12-14h)
- afternoon: (15-17h)
- evening: (18-20h)
- night: (21-23h)

In [ ]:
# Criação da coluna week_day
df_clean["datetime"] = pd.to_datetime(
  df_clean["event_timestamp"],
  unit="us",
  errors="coerce"
)
df_clean['week_day'] = df_clean['datetime'].dt.day_name()

# Criação da coluna hour
df_clean['hour'] = df_clean['datetime'].dt.hour

# Criação da coluna year_num
df_clean['year_num'] = df_clean['datetime'].dt.year

# Criação da coluna week_num
df_clean['week_num'] = df_clean['datetime'].dt.isocalendar().week

# Criação da coluna week_order
df_clean['week_order'] = df_clean['year_num'].astype(str) + '-' + df_clean['week_num'].astype(str)
df_clean['week_order'] = pd.Categorical(df_clean['week_order'],
                                  categories=df_clean['week_order'].unique(),
                                  ordered=True)

# Função para mapear hora → período
def get_period(h):
  if 6 <= h <= 8:
    return 'dawn'
  elif 9 <= h <= 11:
    return 'morning'
  elif 12 <= h <= 14:
    return 'noon'
  elif 15 <= h <= 17:
    return 'afternoon'
  elif 18 <= h <= 20:
    return 'evening'
  elif 21 <= h <= 23:
    return 'night'
  else:
    return 'midnight'

df_clean['period_day'] = df_clean['hour'].apply(get_period)

# Ordenação customizada dos períodos
period_order = ['midnight','dawn','morning','noon','afternoon','evening','night']
df_clean['period_day'] = pd.Categorical(df_clean['period_day'], categories=period_order, ordered=True)


df_clean = df_clean.drop(columns=['datetime'])
# Exibir as primeiras linhas do DataFrame limpo
# df_clean.head()

## Filtragem

### Filtragem dos usuários nos dados limpos

De acordo com o estudo, foram feitas filtragem para treinar os modelos apenas com usuários que tiveram uma interação rica e diversificada com o aplicativo

In [ ]:
print("--- ESTADO INICIAL ---")
print(f"Total de linhas: {df_clean.shape[0]:,}")
print(f"Total de usuários únicos: {df_clean['user_uuid'].nunique():,}")
print("="*60)

# Cópia do DataFrame limpo para aplicar os filtros
df_filtered = df_clean.copy()

# --- Filtro 1: Mínimo de 50 cliques por usuário ---
print("\n>>> APLICANDO FILTRO 1: Mínimo de 50 cliques")
rows_before = df_filtered.shape[0]
users_before = df_filtered['user_uuid'].nunique()

user_clicks = df_filtered.groupby('user_uuid')['event_timestamp'].count()
qualified_users = user_clicks[user_clicks >= 50].index
df_filtered = df_filtered[df_filtered['user_uuid'].isin(qualified_users)]

print(f"   - Linhas removidas: {rows_before - df_filtered.shape[0]:,}")
print(f"   - Usuários removidos: {users_before - df_filtered['user_uuid'].nunique():,}")
print(f"   - Estado atual: {df_filtered.shape[0]:,} linhas e {df_filtered['user_uuid'].nunique():,} usuários.")
print("-"*60)


# --- Filtro 2: Mínimo de 10 locais distintos de clique ---
print("\n>>> APLICANDO FILTRO 2: Mínimo de 10 locais distintos")
rows_before = df_filtered.shape[0]
users_before = df_filtered['user_uuid'].nunique()

user_locations = df_filtered.groupby('user_uuid')['click_location'].nunique()
qualified_users = user_locations[user_locations >= 10].index
df_filtered = df_filtered[df_filtered['user_uuid'].isin(qualified_users)]

print(f"   - Linhas removidas: {rows_before - df_filtered.shape[0]:,}")
print(f"   - Usuários removidos: {users_before - df_filtered['user_uuid'].nunique():,}")
print(f"   - Estado atual: {df_filtered.shape[0]:,} linhas e {df_filtered['user_uuid'].nunique():,} usuários.")
print("-"*60)


# --- Filtro 3: Mínimo de 20 horários distintos de clique ---
print("\n>>> APLICANDO FILTRO 3: Mínimo de 20 horários distintos")
rows_before = df_filtered.shape[0]
users_before = df_filtered['user_uuid'].nunique()

user_hours = df_filtered.groupby('user_uuid')['hour'].nunique()
qualified_users = user_hours[user_hours >= 20].index
df_filtered = df_filtered[df_filtered['user_uuid'].isin(qualified_users)]

print(f"   - Linhas removidas: {rows_before - df_filtered.shape[0]:,}")
print(f"   - Usuários removidos: {users_before - df_filtered['user_uuid'].nunique():,}")
print(f"   - Estado atual: {df_filtered.shape[0]:,} linhas e {df_filtered['user_uuid'].nunique():,} usuários.")
print("-"*60)


# --- Filtro 4: Mínimo de 1000 cliques por usuário ---
print("\n>>> APLICANDO FILTRO 4: Mínimo de 1000 cliques")
rows_before = df_filtered.shape[0]
users_before = df_filtered['user_uuid'].nunique()

user_clicks_1000 = df_filtered.groupby('user_uuid')['event_timestamp'].count()
qualified_users = user_clicks_1000[user_clicks_1000 >= 1000].index
df_filtered = df_filtered[df_filtered['user_uuid'].isin(qualified_users)]

print(f"   - Linhas removidas: {rows_before - df_filtered.shape[0]:,}")
print(f"   - Usuários removidos: {users_before - df_filtered['user_uuid'].nunique():,}")
print(f"   - Estado atual: {df_filtered.shape[0]:,} linhas e {df_filtered['user_uuid'].nunique():,} usuários.")
print("-"*60)


# --- Filtro 5: Mínimo de 3 semanas de utilização ---
print("\n>>> APLICANDO FILTRO 5: Mínimo de 3 semanas de utilização")
rows_before = df_filtered.shape[0]
users_before = df_filtered['user_uuid'].nunique()

user_weeks = df_filtered.groupby('user_uuid')['week_order'].nunique()
qualified_users = user_weeks[user_weeks >= 3].index
df_filtered = df_filtered[df_filtered['user_uuid'].isin(qualified_users)]

print(f"   - Linhas removidas: {rows_before - df_filtered.shape[0]:,}")
print(f"   - Usuários removidos: {users_before - df_filtered['user_uuid'].nunique():,}")
print(f"   - Estado atual: {df_filtered.shape[0]:,} linhas e {df_filtered['user_uuid'].nunique():,} usuários.")
print("="*60)


# --- RESULTADO FINAL ---
print("\n--- ESTADO FINAL ---")
print(f"Total de linhas no dataset final: {df_filtered.shape[0]:,}")
print(f"Total de usuários únicos no dataset final: {df_filtered['user_uuid'].nunique():,}")
print("="*60)

### Converter texto do pictograma pra minúsculo e mantendo apenas últimos cards

In [ ]:
# Convertendo o texto escrito no card para minúsculas
df_filtered['card_written_text'] = df_filtered['card_written_text'].apply(lambda s: s.lower() if isinstance(s, str) else s)

# Filtrando apenas por 'card_is_leaf' = True
df_filtered = df_filtered[df_filtered['card_is_leaf'] == True].copy()
print("\n--- Filtro aplicado: Apenas cards finais ---")
print(f"Total de linhas após filtro: {df_filtered.shape[0]:,}")
print(f"Total de usuários únicos no dataset final: {df_filtered['user_uuid'].nunique():,}")

### Filtrando as colunas importantes

In [ ]:
columns = ['user_uuid', 'click_location', 'card_written_text', 'event_timestamp', 'week_day', 'hour', 'year_num', 'week_num', 'week_order', 'period_day']
df_filtered = df_filtered.loc[:, columns]

In [ ]:
# Salvando o DataFrame final filtrado
path_parquet = '../data/processed/df_filtered.parquet'
df_filtered.to_parquet(path_parquet, index=False)

print(f"DataFrame filtrado salvo com sucesso em: {path_parquet}")

## Divisão de Treino e Teste e clusterização do conjunto de Treino

A coluna 'cluster' foi criada a partir da clusterização com ajuda do algoritmo OPTICS, um método baseado em densidade que ordena os pontos de um conjunto de dados para revelar estruturas de agrupamento. Essa coluna será utilizada como contexto no treinamento

### 1. Preparação dos dados

In [ ]:
# Separar as coordenadas
location_coords = df_filtered['click_location'].astype(str).str.split(',', expand=True)

# Criar as novas colunas numéricas no df_filtered
df_filtered['click_loc_x'] = pd.to_numeric(location_coords[0], errors='coerce')
df_filtered['click_loc_y'] = pd.to_numeric(location_coords[1], errors='coerce')

# Preencher valores nulos com a mediana
df_filtered['click_loc_x'] = df_filtered['click_loc_x'].fillna(df_filtered['click_loc_x'].median())
df_filtered['click_loc_y'] = df_filtered['click_loc_y'].fillna(df_filtered['click_loc_y'].median())

print("Engenharia de features concluída. Novas colunas adicionadas ao df_filtered:")
print(df_filtered[['click_loc_x', 'click_loc_y', 'hour']].head())

### 2. Função para geração dos clusters no Treino e porpagação no Teste

In [ ]:
def gerar_features_cluster(train_df, test_df, feature_cols, optics_params):
  """
  Aplica a clusterização no conjunto de treino e propaga os rótulos para o conjunto de teste.

  Args:
      train_df (pd.DataFrame): DataFrame de treino do usuário.
      test_df (pd.DataFrame): DataFrame de teste do usuário.
      feature_cols (list): Lista de nomes de colunas a serem usadas para a clusterização.
      optics_params (dict): Dicionário com os parâmetros para o algoritmo OPTICS.

  Returns:
      tuple: Uma tupla contendo (train_df_com_cluster, test_df_com_cluster).
  """
  print(f"   - Iniciando clusterização para {len(train_df)} registros de treino...")

  # Copia os DataFrames para evitar warnings de cópia
  train_df_out = train_df.copy()
  test_df_out = test_df.copy()

  # --- Etapa 1: Clusterizar o Conjunto de Treino ---
  features_train = train_df_out[feature_cols]

  # O scaler é treinado APENAS nos dados de treino
  scaler = StandardScaler()
  features_train_scaled = scaler.fit_transform(features_train)

  # O OPTICS é executado APENAS nos dados de treino
  optics = OPTICS(**optics_params)
  train_clusters = optics.fit_predict(features_train_scaled)
  train_df_out['cluster'] = train_clusters

  print(f"   - Finalizado. Encontrados {len(pd.Series(train_clusters).unique())} clusters.")


  # --- Etapa 2: Propagar os Rótulos para o Conjunto de Teste ---
  # O KNN aprende a mapear as features do treino para os clusters do treino
  knn = KNeighborsClassifier(n_neighbors=5)
  knn.fit(features_train_scaled, train_clusters)

  # Usa o MESMO scaler para transformar os dados de teste
  features_test = test_df_out[feature_cols]
  features_test_scaled = scaler.transform(features_test)

  # Prevê os clusters para os dados de teste
  test_clusters = knn.predict(features_test_scaled)
  test_df_out['cluster'] = test_clusters

  return train_df_out, test_df_out

### 3. Preparando os encoders globais

In [ ]:
print("--- Preparando Encoder Global de Usuários ---")
global_encoder_path = '../models/global_encoders'
os.makedirs(global_encoder_path, exist_ok=True)

# O único encoder global agora é o de usuários
all_user_uuids = df_filtered["user_uuid"].unique()
le_user = LabelEncoder().fit(all_user_uuids)
joblib.dump(le_user, os.path.join(global_encoder_path, "label_encoder_user.pkl"))
print(f"LabelEncoder de usuários treinado e salvo. Total de usuários: {len(le_user.classes_)}")

### 4. Divisão, clusterização e encoding por usuário

In [ ]:
# Parâmetros usados para a clusterização
FEATURES_CLUSTER = ['click_loc_x', 'click_loc_y', 'hour']

# Listas para guardar os dataframes processados de cada usuário
train_parts = []
test_parts = []

# Lista para DataFrame que será utilizado para visualização dos clusters
train_parts_for_viz = []

# Ordena o DataFrame para garantir que a divisão por semanas seja consistente
df_filtered_sorted = df_filtered.sort_values(['user_uuid', 'event_timestamp'])
lista_usuarios = df_filtered_sorted['user_uuid'].unique()

for i, user_id in enumerate(lista_usuarios):
    print(f"\n--- Processando user_{i} ---")
    user_model_path = f'../models/user_{i}'
    os.makedirs(user_model_path, exist_ok=True)

    # DataFrame do usuário
    df_u = df_filtered_sorted[df_filtered_sorted['user_uuid'] == user_id]

    # Treinando o LabelEncoder de Cartões antes da divisão, para garantir que ele seja treinado com todos os dados do usuário
    le_card_user = LabelEncoder().fit(df_u["card_written_text"])
    joblib.dump(le_card_user, os.path.join(user_model_path, "label_encoder_card.pkl"))
    print(f"   - Label Encoder de cards treinado e salvo em {user_model_path}")

    # Divisão Treino/Teste
    semanas = sorted(df_u['week_order'].unique())
    if len(semanas) < 3:
        continue
    df_u = df_u.sort_values('week_order')
    weeks_test, weeks_train = semanas[-2:], semanas[:-2]
    df_train, df_test = df_u[df_u['week_order'].isin(weeks_train)].copy(), df_u[df_u['week_order'].isin(weeks_test)].copy()

    if df_train.empty or df_test.empty:
        continue

    # Define min_samples como 3.5% dos dados de treino, com um mínimo de 50
    dynamic_min_samples = max(50, int(len(df_train) * 0.035))
    # Define o min_cluster_size como o dobro, para filtrar clusters menores
    dynamic_min_cluster_size = dynamic_min_samples * 2

    params_optics_user = {
        'min_samples': dynamic_min_samples,
        'xi': 0.05,
        'min_cluster_size': dynamic_min_cluster_size
    }
    print(f"   - Usando parâmetros OPTICS dinâmicos: {params_optics_user}")

    # Clusterização
    df_train, df_test = gerar_features_cluster(df_train, df_test, FEATURES_CLUSTER, params_optics_user)

    # Salvando o DataFrame para visualização dos clusters
    df_train['user_uuid_enc'] = le_user.transform(df_train["user_uuid"])
    train_parts_for_viz.append(df_train)

    print("   - Aplicando Encoders...")
    # Label Encoder de Cartões (aplicado aos dataframes de treino e teste)
    df_train['card_enc'] = le_card_user.transform(df_train["card_written_text"])
    df_test['card_enc'] = le_card_user.transform(df_test["card_written_text"])

    # Label Encoder de Usuário (usando o encoder GLOBAL)
    df_train['user_uuid_enc'] = le_user.transform(df_train["user_uuid"])
    df_test['user_uuid_enc'] = le_user.transform(df_test["user_uuid"])

    # One-Hot Encoding (treinado por usuário)
    cols_ctx = ['period_day', 'week_day', 'cluster']
    ohe_ctx = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

    # Treina o OHE com os dados de treino do usuário e o salva
    ohe_ctx.fit(df_train[cols_ctx])
    joblib.dump(ohe_ctx, os.path.join(user_model_path, "onehot_encoder.pkl"))
    print(f"   - One-Hot Encoder treinado e salvo em {user_model_path}")

    # Transforma ambos os dataframes e cria as novas colunas
    for df_part, name in [(df_train, 'train'), (df_test, 'test')]:
        ctx_encoded = ohe_ctx.transform(df_part[cols_ctx])
        feature_names = ohe_ctx.get_feature_names_out(cols_ctx)

        df_encoded = pd.DataFrame(ctx_encoded, columns=feature_names, index=df_part.index)

        # Remove colunas antigas e junta com as novas
        df_part = df_part.drop(columns=["card_written_text", "user_uuid"] + cols_ctx)
        df_final_user = pd.concat([df_part, df_encoded], axis=1)

        # Salva o resultado final do usuário
        output_path = os.path.join(user_model_path, f"{name}_processed.parquet")
        df_final_user.to_parquet(output_path, compression="snappy")
        print(f"   - Arquivo salvo: {output_path}")

        # Adiciona o DataFrame final do usuário à lista de partes
        if name == 'train':
            train_parts.append(df_final_user)
        else:
            test_parts.append(df_final_user)

print("\n--- Juntando e salvando os resultados finais de todos os usuários ---")
df_train_final = pd.concat(train_parts, ignore_index=True)
df_test_final = pd.concat(test_parts, ignore_index=True)
df_for_viz = pd.concat(train_parts_for_viz, ignore_index=True)

# Salva os arquivos finais
output_path_train = '../data/processed/train_final_processed.parquet'
output_path_test = '../data/processed/test_final_processed.parquet'
output_path_viz = '../data/processed/data_for_visualization.parquet'
df_train_final.to_parquet(output_path_train, index=False, compression="snappy")
df_test_final.to_parquet(output_path_test, index=False, compression="snappy")
df_for_viz.to_parquet(output_path_viz, index=False, compression="snappy")
print(f"   - Arquivo de treino final salvo em: {output_path_train}")
print(f"   - Arquivo de teste final salvo em: {output_path_test}")
print(f"   - Arquivo para visualização salvo em: {output_path_viz}")


print("\n--- Processamento Finalizado para Todos os Usuários ---")
print(f"Tamanho final do treino (todos os usuários): {len(df_train_final)} linhas")
print(f"Tamanho final do teste (todos os usuários): {len(df_test_final)} linhas")